Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# AML Pipeline with DataTranferStep
This notebook is used to demonstrate the use of DataTranferStep in AML Pipeline.

In certain cases, you will need to transfer data from one data location to another. For example, your data may be in Files storage and you may want to move it to Blob storage. Or, if your data is in an ADLS account and you want to make it available in the Blob storage. Built-in **DataTransferStep** can help with such movement of data.

The below example shows how to move data in an ADLS account to the Blob storage.

## AML and Pipeline SDK-specific imports

In [ ]:
import os
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core import Workspace, Run, Experiment
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import AdlaStep
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.core import attach_legacy_compute_target
from azureml.data.stored_procedure_parameter import StoredProcedureParameter, StoredProcedureParameterType
from azureml.pipeline.steps import DataTransferStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Register Datastores

In [ ]:
subscription_id = "<my-subscription-id>"
resource_group = "<my-resource-group>"
store_name = "<my-sotrename>"
tenant_id = "<my-tenant>"
client_id = "<my-client-id>"
client_secret = "<my-client-secret>"

adls_datastore = Datastore.register_azure_data_lake(
    workspace=ws,
    datastore_name='MyAdlsDatastore',
    subscription_id=subscription_id, # subscription id of ADLS account
    resource_group=resource_group, # resource group of ADLS account
    store_name=store_name, # ADLS account name
    tenant_id=tenant_id, # tenant id of service principal
    client_id=client_id, # client id of service principal
    client_secret=client_secret) # the secret of service principal
      
blob_datastore = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name='MyBlobDatastore',
    account_name="<my-account>", # Storage account name
    container_name="<my-container>", # Name of Azure blob container
    account_key="<my-account-key>") # Storage account key

# CLI:
# az ml datastore register-blob -n <datastore-name> -a <account-name> -c <container-name> -k <account-key> [-t <sas-token>]

## Create DataReferences

In [ ]:
adls_datastore = Datastore(workspace=ws, name="MyAdlsDatastore")

# adls
adls_data_ref = DataReference(
    datastore=adls_datastore,
    data_reference_name="adls_test_data",
    path_on_datastore="testdata")

blob_datastore = Datastore(workspace=ws, name="MyBlobDatastore")

# blob data
blob_data_ref = DataReference(
    datastore=blob_datastore,
    data_reference_name="blob_test_data",
    path_on_datastore="testdata")

sql_datastore = Datastore(workspace=workspace, name="firstsqlds")

## sql table
sql_table_data_ref = SqlDataReference(
    datastore=sql_datastore,
    data_reference_name="sql_table_data_ref",
    sql_table="TestData")

## sql query
sql_query_data_ref = SqlDataReference(
    datastore=sql_datastore,
    data_reference_name="sql_query_data_ref",
    sql_query="select top 1 * from TestData")

## stored procedure
sql_proc_data_ref = SqlDataReference(
    datastore=sql_datastore,
    data_reference_name="sql_proc_data_ref",
    sql_stored_procedure="TestStoredProcedure")

## stored procedure with parameters
sql_proc_params_data_ref = SqlDataReference(
    datastore=sql_datastore,
    data_reference_name="sql_proc_params_data_ref",
    sql_stored_procedure="TestStoredProcedureWithParameters",
    sql_stored_procedure_params=[StoredProcedureParameter("firstName", "Joe")])

## Setup Data Factory Account

In [ ]:
data_factory_name = 'adftest'

def get_or_create_data_factory(workspace, factory_name):
    try:
        return DataFactoryCompute(workspace, factory_name)
    except ComputeTargetException as e:
        if 'ComputeTargetNotFound' in e.message:
            print('Data factory not found, creating...')
            provisioning_config = DataFactoryCompute.provisioning_configuration()
            data_factory = ComputeTarget.create(workspace, factory_name, provisioning_config)
            data_factory.wait_for_provisioning()
            return data_factory
        else:
            raise e
            
data_factory_compute = get_or_create_data_factory(ws, data_factory_name)

# CLI:
# Create: az ml computetarget setup datafactory -n <name>
# BYOC: az ml computetarget attach datafactory -n <name> -i <resource-id>

## Create a DataTransferStep

**DataTransferStep** is used to transfer data between Azure Blob, Azure Data Lake Store, and Azure SQL database.

- **name:** Name of module
- **source_data_reference:** Input connection that serves as source of data transfer operation.
- **destination_data_reference:** Input connection that serves as destination of data transfer operation.
- **compute_target:** Azure Data Factory to use for transferring data.
- **allow_reuse:** Whether the step should reuse results of previous DataTransferStep when run with same inputs. Set as False to force data to be transferred again.

Optional arguments to explicitly specify whether a path corresponds to a file or a directory. These are useful when storage contains both file and directory with the same name or when creating a new destination path.

- **source_reference_type:** An optional string specifying the type of source_data_reference. Possible values include: 'file', 'directory'. When not specified, we use the type of existing path or directory if it's a new path.
- **destination_reference_type:** An optional string specifying the type of destination_data_reference. Possible values include: 'file', 'directory'. When not specified, we use the type of existing path or directory if it's a new path.

In [ ]:
transfer_adls_to_blob = DataTransferStep(
    name="transfer_adls_to_blob",
    source_data_reference=adls_data_ref,
    destination_data_reference=blob_data_ref,
    data_factory_compute=data_factory_compute)

transfer_sql_to_blob = DataTransferStep(
    name="transfer_sql_to_blob",
    source_data_reference=sql_proc_params_data_ref,
    destination_data_reference=blob_output_data_ref,
    compute_target=data_factory_compute,
    destination_reference_type='file')

## Build and Submit the Experiment

In [ ]:
pipeline = Pipeline(
    description="data_transfer_101",
    workspace=ws,
    steps=[transfer_adls_to_blob, transfer_sql_to_blob])

pipeline_run = Experiment(ws, "Data_Transfer_example").submit(pipeline)
pipeline_run.wait_for_completion()

### View Run Details

In [ ]:
from azureml.train.widgets import RunDetails
RunDetails(pipeline_run).show()

# Next: Databricks as a Compute Target
To use Databricks as a compute target from AML Pipeline, a DatabricksStep is used. This [notebook](./05.use-databricks-as-compute-target.ipynb) demonstrates the use of DatabricksStep in AML Pipeline.